In [7]:
words = open('names.txt', 'r').read().splitlines()
chars = sorted(list(set("".join(words))))
stoi = {c:i+1 for i,c in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [139]:
# create training set
# we want to predict bigram ch1,ch2  
#x=ch1 (what we take as input to nn)
#y=ch2 (what we want to predict)

xs = []
ys = []
for word in words:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        xs.append(stoi[ch1])
        ys.append(stoi[ch2])

# tensor need one hot / (float or int..) not strs..
# tensor.Tensor float, tensor.tensor infers dtype
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print("Number of examples: ", num)

Number of examples:  228146


In [ ]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g, requires_grad=True) # drawn from normal dist. with mean=0

In [146]:
# grad desc
alpha = 50
for k in range(100):
    # each row in xenc @ W gives us probabilities of what comes after x_i for each of 27 possible next char. 
    # lets interpret it as log counts, then we take exp (to give us bet 0-1 and then we can take softmax
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W # log-counts
    counts = (xenc @ W).exp()
    probs = counts / counts.sum(1, keepdims=True) # probs for next char given example
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean() # reg 1/N (W^2)
    print(loss.item())
    #backward pass
    W.grad = None # set to zero
    loss.backward() #pytorch keeps computational graph
    #update
    W.data += -alpha * W.grad

3.76861834526062
3.3788065910339355
3.161090850830078
3.0271859169006348
2.9344842433929443
2.867231607437134
2.8166542053222656
2.777146339416504
2.7452542781829834
2.7188305854797363
2.696505546569824
2.6773719787597656
2.6608054637908936
2.6463515758514404
2.633665084838867
2.622471570968628
2.6125476360321045
2.6037068367004395
2.595794439315796
2.5886809825897217
2.582256317138672
2.5764293670654297
2.5711236000061035
2.566272735595703
2.5618226528167725
2.5577261447906494
2.5539445877075195
2.550442695617676
2.5471930503845215
2.5441696643829346
2.5413525104522705
2.538722038269043
2.536262035369873
2.5339579582214355
2.531797409057617
2.5297679901123047
2.527860164642334
2.526063919067383
2.5243709087371826
2.522773265838623
2.52126407623291
2.519836664199829
2.5184855461120605
2.517204999923706
2.515990972518921
2.5148372650146484
2.5137410163879395
2.512698173522949
2.511704444885254
2.5107579231262207
2.509855031967163
2.5089924335479736
2.5081682205200195
2.5073797702789307


In [ ]:
# What do we expect it to converge to?
# Lower NLL better.. but whats the best we can do? That is the probability of observing the data we are given, that is the maximum likelihood
# 2.47 loss is the same loss we got when we empirical counted and calculated our NLL. that is the prob of our observed data.
# The NN method scales much better.. dont need to load all the 

# The weights is exactly the logits of the counts that we counted!! same 27,27 arr
# We did smoothing in the other e.g add 1 laplace to aovid 0 prob for the log. 
# The more add add, the more 'smoothed' the distribution because and the predictions are same
# Similar idea happens when all Weights are 0. exp that -> 1 -> all probs equal chance
# weights closer to 0 max entropy zero implies "no opinion" or "maximum uncertainty.", helps prevent overfitting
#  Small weights force the model to be smoother and more general.
# that is L1/L2 regulization

In [151]:
# samping from NN model
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    out = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

cexze.
momasurailezityha.
konimittain.
llayn.
ka.
